# Deploy an MLflow model with SageMaker

## Install MLflow

In [ ]:
!pip install -q mlflow==1.28.0

## Setup environment

In [ ]:
import json
import boto3
import mlflow
import sagemaker
import pandas as pd
import mlflow.sagemaker
from sklearn.datasets import load_boston

# name of the AWS region to which to deploy the application
region = sagemaker.Session().boto_region_name
# we are using the notebook instance role for training in this example
role = sagemaker.get_execution_role() 
# uri of your remote mlflow server
tracking_uri = '<YOUR MLFLOW SERVER URI>' 
# set remote mlflow server
mlflow.set_tracking_uri(tracking_uri)

## Build MLflow docker image to serve the model with SageMaker 

In [ ]:
!mlflow sagemaker build-and-push-container

In [ ]:
# URL of the ECR-hosted Docker image the model should be deployed into
image_uri = '<YOUR mlflow-pyfunc ECR IMAGE URI>'

## Deploy a SageMaker endpoint with our scikit-learn model

In [ ]:
endpoint_name = 'boston-housing'
# The location, in URI format, of the MLflow model to deploy to SageMaker.
model_uri = 'models:/<model_name>/<model_version>'

In [ ]:
mlflow.sagemaker.deploy(
    mode='create',
    app_name=endpoint_name,
    model_uri=model_uri,
    image_url=image_uri,
    execution_role_arn=role,
    instance_type='ml.m5.xlarge',
    instance_count=1,
    region_name=region
)

## Predict

In [ ]:
# load boston dataset
data = load_boston()
df = pd.DataFrame(data.data, columns=data.feature_names)

In [ ]:
runtime= boto3.client('runtime.sagemaker')
# predict on the first row of the dataset
payload = df.iloc[[0]].to_json(orient="split")

runtime_response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=payload)
result = json.loads(runtime_response['Body'].read().decode())
print(f'Payload: {payload}')
print(f'Prediction: {result}')

## Delete endpoint

In [ ]:
mlflow.sagemaker.delete(app_name=endpoint_name, region_name=region)